In [1]:
# Import necessary libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import os
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\anasn\miniconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\anasn\miniconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\anasn\miniconda3\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\anasn\miniconda3\Lib\site-

In [3]:

# Define constants
COLUMN_NAMES = [
    "id", "category_label", "unknown1", "unknown2", "unknown3",
    "category", "json_content", "date", "policy_url", "policy_text"
]

# Function to process annotations from the CSV file
def process_annotations(csv_file_path):
    """Load and process annotations from a CSV file.
    Args:
        csv_file_path (str): Path to the CSV file.
    Returns:
        list: List of dictionaries with 'id', 'category', and 'policy_text'.
    """
    data = pd.read_csv(csv_file_path, names=["id", "category", "policy_text"])
    annotations = [
        {"id": row["id"], "category": row["category"], "policy_text": row["policy_text"]}
        for _, row in data.iterrows()
    ]
    return annotations

# Function to extract policy text from an HTML file
def extract_policy_text(html_file_path):
    """Extract all visible text from an HTML file.
    Args:
        html_file_path (str): Path to the HTML file.
    Returns:
        str: Extracted text.
    """
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    policy_text = ' '.join(soup.stripped_strings)
    return policy_text

# Provide file paths
csv_file_path = r'./content/20_theatlantic.com.csv'
html_file_path = r'./content/20_theatlantic.com.html'

# Call functions
annotations = process_annotations(csv_file_path)
policy_text = extract_policy_text(html_file_path)

print(annotations)
print(policy_text)

[{'id': '{"Other Type": {"endIndexInSegment": 762, "startIndexInSegment": 100, "selectedText": "At the Atlantic Monthly Group, Inc. (\\"The Atlantic\\"), we want you to enjoy and benefit from our websites and online services secure in the knowledge that we have implemented fair information practices designed to protect your privacy. Our privacy policy is applicable to The Atlantic, and The Atlantics affiliates and subsidiaries whose websites, mobile applications and other online services are directly linked (the Sites). The privacy policy describes the kinds of information we may gather during your visit to these Sites, how we use your information, when we might disclose your personally identifiable information, and how you can manage your information.", "value": "Introductory/Generic"}}', 'category': '1/1/15', 'policy_text': 'http://www.theatlantic.com/privacy-policy/'}, {'id': '{"Other Type": {"endIndexInSegment": 762, "startIndexInSegment": 100, "selectedText": "At the Atlantic Mont

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define label mapping
label_mapping = {
    "better": 0,
    "not better": 1
}
class PrivacyPolicyDataset(Dataset):
    """Dataset for preparing privacy policies for fine-tuning."""
    def __init__(self, annotations, label_mapping):
        self.annotations = annotations
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.label_mapping = label_mapping
        self.data = self._prepare_data()

    def _prepare_data(self):
        """Prepare data by encoding text and labels."""
        data = []
        for annotation in self.annotations:
            text = annotation["policy_text"]
            category = annotation["category"]
            label = self.label_mapping.get(category, -1)
            data.append((text, label))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]
        text = str(text)
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\anasn\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anasn\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:

def fine_tune_model_optimized(dataset, tokenizer, output_dir="./fine_tuned_model"):
    """Fine-tune a pre-trained BERT model on the given dataset.
    Args:
        dataset (PrivacyPolicyDataset): Dataset for training and evaluation.
        tokenizer (AutoTokenizer): Tokenizer for the model.
        output_dir (str): Directory to save the fine-tuned model.
    Returns:
        AutoModelForSequenceClassification: Fine-tuned model.
    """
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(label_mapping)
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        save_total_limit=2,
        fp16=True,
        dataloader_num_workers=4
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=dataset,
        tokenizer=tokenizer,
    )

    trainer.train()
    model.save_pretrained(output_dir)
    dataset.tokenizer.save_pretrained(output_dir)
    return model


In [7]:
def predict_policy(policy_text, model, tokenizer, label_mapping):
    """Predict the category of a given policy text.
    Args:
        policy_text (str): The text of the policy.
        model (AutoModelForSequenceClassification): Fine-tuned model.
        tokenizer (AutoTokenizer): Tokenizer for the model.
        label_mapping (dict): Mapping of labels to indices.
    Returns:
        str: Predicted category label.
    """
    inputs = tokenizer(
        policy_text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities).item()
    category = {idx: label for label, idx in label_mapping.items()}
    return category.get(predicted_class, "Unknown")


# Prepare dataset
csv_file_path = r"./content/20_theatlantic.com.csv"
annotations = process_annotations(csv_file_path)

# Create a list of policy texts
policy_texts = [annotation["policy_text"] for annotation in annotations if annotation["policy_text"]]

# Initialize the model, tokenizer, and label mapping
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
label_mapping = {
    "better": 0,
    "not better": 1
}

# Make predictions
predicted_categories = []
for policy_text in policy_texts:
    predicted_category = predict_policy(policy_text, model, tokenizer, label_mapping)
    predicted_categories.append(predicted_category)

# Print the predicted categories
print(predicted_categories)


# Split dataset into training and testing sets
train_size = int(0.8 * len(policy_texts))
test_size = len(policy_texts) - train_size
train_texts, test_texts = policy_texts[:train_size], policy_texts[train_size:]

# Create datasets for training and testing
train_dataset = PrivacyPolicyDataset(train_texts, label_mapping)
test_dataset = PrivacyPolicyDataset(test_texts, label_mapping)

# Fine-tune model
output_dir = "./fine_tuned_model"
model = fine_tune_model_optimized(train_dataset, tokenizer, output_dir, label_mapping)

# Evaluate model on test set
test_pred = []
test_labels = []
for policy_text in test_texts:
    pred = predict_policy(policy_text, model, tokenizer, label_mapping)
    test_pred.append(pred)
    test_labels.append(0)

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, [label_mapping.get(label, -1) for label in test_pred])
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(test_labels, [label_mapping.get(label, -1) for label in test_pred]))


# Use the model to make predictions
html_file_path = r'./content/20_theatlantic.com.html'
policy_text = extract_policy_text(html_file_path)
predicted_category = predict_policy(policy_text, model, tokenizer, label_mapping)
print("Predicted category:", predicted_category)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: 